In [1]:
##### EGE BİRCAN  20950
##### ATAKAN ATİK 20537

##### CS412 MACHINE LEARNING PROJECT


import numpy as np 
import csv
import pandas as pd

# Reading csv files from according paths
testData = pd.read_csv("/home/egebircan/Desktop/loansTest.csv", header=None, skiprows=1)
trainData = pd.read_csv("/home/egebircan/Desktop/loansTrain.csv", header=None, skiprows=1)

# Analyzing the shapes to control if they are fittable
print('Training Data:', trainData.shape)
print('Testing Data:', testData.shape)


# In training data there were 1 million rows and in test data there were 300k rows. And in training data there
# were at most 130k nan values for a single column. Which is not too much with respect to 1 millon lines. Also the
# same situation occured with test data. So instead of filling the nan values with mean or trying to edit them and
# sacrifice our processing power and time, we chose filling them with 0's to gain time and processing power. Since
# nan values were not too many with respect to the data sizes, we thought it is an efficient decision.
trainData.fillna(0, inplace=True)
testData.fillna(0, inplace=True)


# Holding a dictionary to fill with the unique values inside data columns
dictionary = dict()
counter = 0


# Since our algorithm Decision Tree cannot deal with strings, we had encode them somehow. We decided to use the built
# in encoder for python pandas but then we realised that pandas encoder encodes train and test data different. Since each
# of them has different unique values and some values that occur in train data does not occur in test data(or vice-versa)
# So to overcome this problem we decided to hold a dictionary obtained from training data, and apply the key's values to
# test data. This way both of the datas would be encoded using the same key value pairs.

# Obtaining the dictionary from training data:
for x in range (0, 1000000):
    if not trainData.iloc[x,1] in dictionary.keys(): 
        dictionary[trainData.iloc[x,1]] = counter
        counter = counter + 1 
    if not trainData.iloc[x,2] in dictionary.keys():
        dictionary[trainData.iloc[x,2]] = counter
        counter = counter + 1     
    if not trainData.iloc[x,5] in dictionary.keys():
        dictionary[trainData.iloc[x,5]] = counter
        counter = counter + 1 
    if not trainData.iloc[x,7] in dictionary.keys():
        dictionary[trainData.iloc[x,7]] = counter
        counter = counter + 1 

        
# Here we realised that training data does not have credit grade G but the test data has it. So we decided to add grade 
# G hardcoded to save from time and efficiency. After adding it to the dictionary we gave it a value using counter variable.
dictionary["G"] = counter + 1
counter = counter + 1


# Traversing the training data and obtaining state codes were not efficient since there are only a small 
# limited number of them. We decided to add them to the dictionary and gave values hardcoded.
stateCodes = ["AS", "DC", "FM", "GU", "MH", "MP", "PW", "PR", "VI", "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
for x in stateCodes:
    dictionary[x] = counter + 1
    counter = counter + 1
print(dictionary)
trainAttributes = list()
trainResult = list()
testAttributes = list()


# Since we finished encoding our data using the dictionaries, it is time to form data arrays obtained from encoded data.
# in this algorithm we get the value from the data, check our dictionary if the data obtained is a key in the dictionary.
# if it exists in the dictionary that means the value is string and it is encoded. We use the key's value in the actual
# data array so that our actual data frame will be encoded too. If the value obtained from the data does not exist in the
# dictionary it means that the value is integer/float. So we just add it to data frame just as it is.

for x in range (0, 1000000):
    attributeRowList = list()
    # in the if lines below, we got rid of the low correlation columns to make our machine learning algorithm
    # more accurate and save from running time and gain efficiency.
    for y in range (0, 58):
        if y != 3 and y != 4 and y != 8 and y != 9 and y != 13 and y != 20:
            if trainData.iloc[x,y] in dictionary.keys(): 
                #print("dictionary'de kayıtlı")
                attributeRowList.append(dictionary[trainData.iloc[x,y]])
            else:
                attributeRowList.append(trainData.iloc[x,y])
        
    trainAttributes.append(attributeRowList)
    trainResult.append(trainData.iloc[x,58])

#print(trainResult)

for x in range (0, 303605):
    testAttributeRowList = list()
    for y in range (1, 59):
        if y != 4 and y != 5 and y != 9 and y != 10 and y != 14 and y != 21:
            if testData.iloc[x,y] in dictionary.keys(): 
                #print("dictionary'de kayıtlı")
                testAttributeRowList.append(dictionary[testData.iloc[x,y]])
            else:
                testAttributeRowList.append(testData.iloc[x,y])

    testAttributes.append(testAttributeRowList)    
    


X_train = trainAttributes
y_train = trainResult
X_test = testAttributes

# We chose decision tree algorithm because the data we are working on had some string values. And uniqueness of these
# values were very low. For example home_ownership column consists of string values but the values are only 3 types.
# rent, mortgage and own. verification_status column had also the same thing. It only consists of 3 types that are 
# verified, not verified, source verified. Since these are the qualities of the data, we thought it would be easy
# and efficient to form different classes out of these string variables after encoding.

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)

print(y_predict)   

idList = list()
for x in range (0,303605):
    idList.append(x)

rows = zip(idList, y_predict)

import csv

# Outputting the rows to a csv file as the sample output.

with open("/home/egebircan/Desktop/loanPredictions.csv", "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

Training Data: (1000000, 59)
Testing Data: (303605, 59)
{' 60 months': 0, 'E': 1, 'RENT': 2, 'Verified': 3, 'D': 4, 'MORTGAGE': 5, 'Source Verified': 6, ' 36 months': 7, 'B': 8, 'Not Verified': 9, 'A': 10, 'F': 11, 'C': 12, 'OWN': 13, 'G': 19, 'ANY': 15, 'OTHER': 16, 'NONE': 17, 'AS': 20, 'DC': 21, 'FM': 22, 'GU': 23, 'MH': 24, 'MP': 25, 'PW': 26, 'PR': 27, 'VI': 28, 'AL': 29, 'AK': 30, 'AZ': 31, 'AR': 32, 'CA': 33, 'CO': 34, 'CT': 35, 'DE': 36, 'FL': 37, 'GA': 38, 'HI': 39, 'ID': 40, 'IL': 41, 'IN': 42, 'IA': 43, 'KS': 44, 'KY': 45, 'LA': 46, 'ME': 47, 'MD': 48, 'MA': 49, 'MI': 50, 'MN': 51, 'MS': 52, 'MO': 53, 'MT': 54, 'NE': 55, 'NV': 56, 'NH': 57, 'NJ': 58, 'NM': 59, 'NY': 60, 'NC': 61, 'ND': 62, 'OH': 63, 'OK': 64, 'OR': 65, 'PA': 66, 'RI': 67, 'SC': 68, 'SD': 69, 'TN': 70, 'TX': 71, 'UT': 72, 'VT': 73, 'VA': 74, 'WA': 75, 'WV': 76, 'WI': 77, 'WY': 78}
[1 1 1 ... 1 0 1]
